<a href="https://colab.research.google.com/github/omier/NLP-final-project/blob/master/NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init Notebook

In [1]:
!git clone https://github.com/omier/NLP-final-project.git

Cloning into 'NLP-final-project'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [2]:
!pip install -qq transformers

     |████████████████████████████████| 1.8MB 11.0MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import plotly.express as px
import pprint

In [4]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Load Data

In [6]:
df = pd.read_csv("NLP-final-project/Data/Combined_News_DJIA.csv")

In [7]:
df = df.drop(['Date'], axis=1)
tops = df.columns[1:]

df['headline'] = df[tops[:1]].apply(lambda row:'.'.join(row.values.astype(str)), axis=1)
df = df.drop(tops, axis=1)

df['label'] = df['Label']
df = df.drop(['Label'], axis=1)

# remove special characters
df = df.replace('b\"|b\'|\\\\|\\\"', '', regex=True)

df.head()

,headline,label
0,Georgia 'downs two Russian warplanes' as count...,0
1,Why wont America and Nato help us? If they won...,1
2,Remember that adorable 9-year-old who sang at ...,0
3,U.S. refuses Israel weapons to attack Iran: r...,0
4,All the experts admit that we should legalise ...,1


In [8]:
max_len = 0
for i, r in df.iterrows():
  max_len = max(max_len, len(r["headline"]))

MAX_LEN = max_len

In [9]:
class HeadlinesStocksDataset(Dataset):
  def __init__(self, headlines, labels, tokenizer, max_len):
    self.headlines = headlines
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.headlines)

  def __getitem__(self, item):
    headline = str(self.headlines[item])
    label = self.labels[item]
    encoding = self.tokenizer.encode_plus(
      headline,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'headline': headline,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'labels': torch.tensor(label, dtype=torch.long)
    }

In [10]:
df_train, df_test = train_test_split(
  df,
  test_size=0.8,
  random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

In [11]:
BATCH_SIZE = 8

def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = HeadlinesStocksDataset(
    headlines=df['headline'].to_numpy(),
    labels=df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size
  )

# Helpers

In [12]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler):
  losses = []
  correct_predictions = 0

  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    labels = d["labels"].float().to(device)

    # call model
    outputs = torch.squeeze(model(input_ids=input_ids, attention_mask=attention_mask))

    # for accuracy calculation
    predicted_labels = (outputs > 0.5).float() * 1
    correct_predictions += torch.sum(predicted_labels == labels)

    # calculate and save loss
    loss = loss_fn(outputs, labels)
    losses.append(loss.item())

    # optimizer step
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  
  return {'accuracy': correct_predictions.item() / len(data_loader.dataset),
          'loss': np.mean(losses)}

In [13]:
def eval_model(model, loss_fn, sets=['validation']):
  with torch.no_grad():
    sets_metrics = dict()

    for set_name, dataloader in data_loaders.items():
      if set_name in sets:
        losses = []
        correct_predictions = 0
  
        for d in dataloader:
          input_ids = d["input_ids"].to(device)
          attention_mask = d["attention_mask"].to(device)
          labels = d["labels"].float().to(device)

          # call model
          outputs = torch.squeeze(model(input_ids=input_ids, attention_mask=attention_mask))

          # for accuracy calculation
          predicted_labels = (outputs > 0.5).float() * 1
          correct_predictions += torch.sum(predicted_labels == labels)

          # calculate and save loss
          losses.append(loss_fn(outputs, labels).item())

        sets_metrics[set_name] = {'accuracy': correct_predictions.item() / len(dataloader.dataset),
                                  'loss': np.mean(losses)}

    return sets_metrics


In [14]:
def train(model, n_epochs, train_data_loader, loss_fn, optimizer, scheduler):
  history = []

  for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/{n_epochs} Metrics')
    
    current_metrics = { 'train': 
      train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        scheduler
      ) 
    }
    current_metrics.update(eval_model(model, loss_fn))

    pprint.pprint(current_metrics, indent=4)
    print('-' * 10)

    history.append(current_metrics)

  return history

In [15]:
def plot(history):
  metrics_map = dict()

  for e_sets in history:
    for set_name, set_metrics in e_sets.items():
      for metric_name, metric_value in set_metrics.items():

        if metric_name not in metrics_map:
          metrics_map[metric_name] = dict()
        if set_name not in metrics_map[metric_name]:
          metrics_map[metric_name][set_name] = []

        metrics_map[metric_name][set_name].append(metric_value)
      
  for metric_name, sets in metrics_map.items():
    df = None
    for set_name, set_metrics in sets.items():
      size = len(set_metrics)
      if df is None:
        df = pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size})
      else:
        df = df.append(pd.DataFrame({"epoch": np.linspace(1, size, size),
                      metric_name: set_metrics,
                      "set": [set_name] * size}), ignore_index=True)

    fig = px.line(df, x="epoch", y=metric_name, line_group="set", title=f"epoch {metric_name} per dataset", color="set", hover_name="set")
    fig.show()

# Import BERT

In [16]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [17]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [20]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [21]:
bert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

# BERT Transfer Learning

In [18]:
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [19]:
data_loaders = {'train': train_data_loader, 'validation': val_data_loader, 'test': test_data_loader}

In [22]:
class SentimentClassifier(nn.Module):
  def __init__(self, bert):
    super(SentimentClassifier, self).__init__()
    self.bert = bert
    self.drop = nn.Dropout(p=0.3)
    self.classifier = nn.Sequential(nn.Linear(self.bert.config.hidden_size, 1),
                                    nn.Sigmoid())

  
  def forward(self, input_ids, attention_mask):
    pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    ).pooler_output

    droped_output = self.drop(pooled_output)
     
    return self.classifier(droped_output)  


In [23]:
model = SentimentClassifier(bert_model).to(device)

In [24]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.BCELoss().to(device)

In [25]:
%%time
history = train(model, EPOCHS, train_data_loader, loss_fn, optimizer, scheduler)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/10 Metrics


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



{   'train': {'accuracy': 0.491183879093199, 'loss': 0.723464601635933},
    'validation': {'accuracy': 0.48743718592964824, 'loss': 0.7077741795778274}}
----------
Epoch 2/10 Metrics
{   'train': {'accuracy': 0.5541561712846348, 'loss': 0.6850408852100373},
    'validation': {'accuracy': 0.4824120603015075, 'loss': 0.7397498154640197}}
----------
Epoch 3/10 Metrics
{   'train': {'accuracy': 0.6775818639798489, 'loss': 0.5582800634205342},
    'validation': {'accuracy': 0.5414572864321608, 'loss': 0.9233795392513275}}
----------
Epoch 4/10 Metrics
{   'train': {'accuracy': 0.8841309823677582, 'loss': 0.30995909255929294},
    'validation': {'accuracy': 0.457286432160804, 'loss': 2.8850132995843887}}
----------
Epoch 5/10 Metrics
{   'train': {'accuracy': 0.9596977329974811, 'loss': 0.18698361489688978},
    'validation': {'accuracy': 0.4949748743718593, 'loss': 2.8666506663337348}}
----------
Epoch 6/10 Metrics
{   'train': {'accuracy': 0.9949622166246851, 'loss': 0.02138035078998655},

In [26]:
plot(history)

In [28]:
eval_model(model, loss_fn, sets=['test'])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning:

The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).



{'test': {'accuracy': 0.4748743718592965, 'loss': 3.287807224988937}}